In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import cross_validation
import xgboost as xgb
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
w = pd.read_csv('tmp/engineered_features.csv', index_col='file')

In [3]:
w.info()

<class 'pandas.core.frame.DataFrame'>
Index: 403796 entries, 3093804_raw_html.txt to 4059503_raw_html.txt
Columns: 388 entries, sponsored to 276
dtypes: float64(388)
memory usage: 1.2+ GB


In [4]:
tr = w[pd.notnull(w.sponsored)]
tr = tr.fillna(0)
tr.info()
#tr[:5].T

<class 'pandas.core.frame.DataFrame'>
Index: 337024 entries, 3093804_raw_html.txt to 3198940_raw_html.txt
Columns: 388 entries, sponsored to 276
dtypes: float64(388)
memory usage: 1000.2+ MB


In [5]:
trl = tr.sponsored
del tr['sponsored']
#trl[:5]

In [6]:
te = w[pd.isnull(w.sponsored)]
del te['sponsored']
te = te.fillna(0)
#te.info()
#te[:5].T

In [7]:
#m = RandomForestClassifier(n_estimators=10, n_jobs=-1)
#m = KNeighborsClassifier(n_neighbors=5)
#m = SVC(probability=True)

In [8]:
m = xgb.XGBClassifier(max_depth=25, n_estimators=600, learning_rate=0.04, gamma=0.07, min_child_weight=3)

In [9]:
#scores = cross_validation.cross_val_score(m, tr, trl, cv=3, scoring='roc_auc')

In [10]:
# RandomForestClassifier(n_estimators=500, max_depth=30, oob_score=True, n_jobs=-1):
#     array([ 0.90357458,  0.90298073,  0.90189454])
# KNeighborsClassifier(n_neighbors=5):
#     array([ 0.67784316,  0.68113898,  0.6737584 ])
#scores

In [11]:
m.fit(tr, trl)

XGBClassifier(base_score=0.5, colsample_bytree=1, gamma=0.07,
       learning_rate=0.04, max_delta_step=0, max_depth=25,
       min_child_weight=3, missing=None, n_estimators=600, nthread=-1,
       objective='binary:logistic', seed=0, silent=True, subsample=1)

In [12]:
preds = DataFrame(m.predict_proba(te))[1]
sub = DataFrame({'file': te.index, 'sponsored': preds})
sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66772 entries, 0 to 66771
Data columns (total 2 columns):
file         66772 non-null object
sponsored    66772 non-null float32
dtypes: float32(1), object(1)
memory usage: 1.3+ MB


In [13]:
sub.to_csv('output/prec_rec_count_okapi_xgboost_param_opt_01.csv', index=False)